#### Construction de la base d'apprentissage

In [1]:
import numpy as np
import pandas as pd
import csv
from random import shuffle

In [2]:
def transformer_sequence(sequence,long):
    nucleotides = np.zeros((4,int(max(long,25))))
    cnt = 0
    for lettre in sequence:
        if lettre=='a':
            nucleotides[0,cnt] = 1
            cnt = cnt+1
        elif lettre=='c':
            nucleotides[1,cnt] = 1
            cnt = cnt+1
        elif lettre=='g':
            nucleotides[2,cnt] = 1
            cnt = cnt+1
        else:
            nucleotides[3,cnt] = 1
            cnt = cnt+1
    return nucleotides

In [3]:
def transformer_structure(structure):
    long = len(structure)
    a = np.zeros(25)
    a[:long] = structure
    return a

In [4]:
def traduire(seq,strand):
    nucleotides = ''
    if strand:
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'a'
            elif lettre=='C':
                nucleotides=nucleotides+'c'
            elif lettre=='G':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'    
    else:
        string = ''
        for i in seq:
            string = i + string
        seq = string
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'u'
            elif lettre=='C':
                nucleotides=nucleotides+'g'
            elif lettre=='G':
                nucleotides=nucleotides+'c'
            else:
                nucleotides=nucleotides+'a'
    return nucleotides

### Réseau de neurones basé sur iDeep

In [5]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
#from keras.optimizers import kl_divergence
from sklearn import svm, grid_search
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib 
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse

from keras.layers import Merge

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [7]:
from keras.callbacks import EarlyStopping

In [8]:
from keras.layers.convolutional import AveragePooling2D

In [9]:
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+ fp)
    sensitivity = float(tp)/ (tp+fn)
    specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 


### Pretraining

In [10]:
# pretraining

from numpy import random

bdd = np.zeros((50000,35,35,1))
B = np.zeros((6,6))
B[3,3]=1
for i in range(25000):
    a = np.random.randint(2)
    if a==0:
        z=random.randint(2)
        k=random.randint(2)
        bdd[i,k:6+k,:6,0] = np.eye(6)-z*B
        k=k+random.randint(2)
        bdd[i,6+k:12+k,6:12,0] = np.eye(6)
        k=k+random.randint(2)
        bdd[i,12+k:18+k,12:18,0] = np.eye(6)-z*B
        k=k+random.randint(2)
        bdd[i,18+k:24+k,18:24,0] = np.eye(6)
        k=k+random.randint(2)
        bdd[i,24+k:30+k,24:30,0] = np.eye(6)-z*B
    elif a==1: 
        z=random.randint(2)
        k=random.randint(2)
        bdd[i,29-k:35-k,29:35,0] = np.eye(6)-z*B
        k=k+random.randint(2)
        bdd[i,23-k:29-k,23:29,0] = np.eye(6)
        k=k+random.randint(2)
        bdd[i,17-k:23-k,17:23,0] = np.eye(6)-z*B
        k=k+random.randint(2)
        bdd[i,11-k:17-k,11:17,0] = np.eye(6)
        k=k+random.randint(2)
        bdd[i,5-k:11-k,5:11,0] = np.eye(6)-z*B
    b = random.randint(6)
    bruit = np.zeros((35,35))
    #bruit[:12+b,23-b:] = random.randint(0,2,(12+b,12+b))
    #bruit[23-b:,:12+b] = random.randint(0,2,(12+b,12+b))
    #bruit[:12+b,23-b:] = (random.rand(12+b,12+b)*3/4+0.53125).astype(int)
    #bruit[23-b:,:12+b] = (random.rand(12+b,12+b)*3/4+0.53125).astype(int)
    for c in range(2,35):
        bruit = np.diag((random.rand(35-c)*3/4+0.53125).astype(int),c)
        bdd[i,:,:,0] = bdd[i,:,:,0] + bruit
        bruit = np.diag((random.rand(35-c)*3/4+0.53125).astype(int),-c)
        bdd[i,:,:,0] = bdd[i,:,:,0] + bruit
    
for i in range(12500,25000):
    p = np.zeros((35,35))
    for j in range(35):        
        p[:,j] = bdd[i,:,35-j-1,0]
    bdd[i,:,:,0] = p
        
for i in range(25000,50000):
    bdd[i,:,:,0] = np.random.rand(35,35)*3/4
    for j in range(35):
        for k in range(35):
            bdd[i,j,k,0] = round(bdd[i,j,k,0])

In [11]:
for i in range(25000):
    for k in range(35):
        for q in range(35):
            bdd[i,k,q,0] = bdd[i,k,q,0]%2

In [12]:
# pretraining

nb_train = 40000
nb_val = 2000
nb_test = 2000

indices = np.arange(len(bdd))
shuffle(indices)
bdd = bdd[indices]
labels = np.zeros(50000)
labels[:25000] = np.ones(25000)
labels = labels[indices]

training = []
training.append(bdd[:nb_train])
validation=[]
validation.append(bdd[nb_train:nb_train+nb_val])
testing=[]
testing.append(bdd[-nb_test:])
bdd=[]
y = labels[:nb_train]
y = keras.utils.np_utils.to_categorical(y,2)
val_y = labels[nb_train:nb_train+nb_val]
val_y = keras.utils.np_utils.to_categorical(val_y,2)
true_y = labels[-nb_test:]
indices=[]

In [14]:
def get_cnn_pretraining():
    k = 6
    nbfilter = 24
    # init_weights
    I = np.eye(k)
    M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
    I2 = np.zeros((k,k))
    M2 = np.zeros((k,k))
    for j in range(k):
        I2[:,j] = I[:,k-j-1]
        M2[:,j] = M[:,k-j-1]        
    W = np.zeros((k,k,1,nbfilter))
    W[:,:,0,0] = I
    W[:,:,0,1] = I2
    W[:,:,0,2] = M
    W[:,:,0,3] = M2
    for j in range(4,nbfilter):
        W[:,:,0,j] = np.random.randn(k,k)*0.2        
    # cnn
    print('configure cnn network')
 
    model = Sequential()
    model.add(Conv2D(filters = nbfilter, kernel_size=(k,k), padding='valid', input_shape=(35,35,1),strides=(1,1),weights=[W,np.zeros(nbfilter)]))
    #model.add(BatchNormalization())
    #model.add(Activation('sigmoid'))
    #model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(3,3)))
    print(model.output_shape)
    nbfilter2 = 4
    k2 = 7
    #
    I = np.eye(k2)
    M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
    I2=np.zeros((k2,k2))
    M2=np.zeros((k2,k2))
    for j in range(k2):
        I2[:,j] = I[:,k2-j-1]
        M2[:,j] = M[:,k2-j-1]   
    Z = np.zeros((k2,k2,nbfilter,nbfilter2))
    for u in range(nbfilter):
        Z[:,:,u,0] = I
        Z[:,:,u,1] = I2
        Z[:,:,u,2] = M
        Z[:,:,u,3] = M2
        for p in range(4,nbfilter2):
            Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            
    #
    model.add(Conv2D(filters = nbfilter2, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter2)]))
    model.add(BatchNormalization())

    print(model.output_shape)
    
    model.add(Flatten())
        
    return model

In [16]:
def network_pretraining():
    
    print('training', nb_train)
    
    training_net=get_cnn_pretraining()

    model = Sequential()
    model.add(training_net)
    model.add(Dense(64,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
        
    earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
    print('model training')
    model.fit(training[0], y, batch_size=16, epochs=2, verbose=1, validation_data=(validation[0], val_y), callbacks=[earlystopper])
    
    # test       
    print('predicting')            
    predictions = model.predict_proba(testing[0])[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])
    
    return model

In [17]:
pretrain = network_pretraining()

('training', 40000)
configure cnn network
(None, 10, 10, 24)
(None, 4, 4, 4)
model training
Train on 40000 samples, validate on 2000 samples
Epoch 1/2
40000/40000 [==============================] - 23s 565us/step - loss: 0.2195 - val_loss: 0.6823
Epoch 2/2
40000/40000 [==============================] - 21s 537us/step - loss: 0.1394 - val_loss: 0.0665
predicting
[9.9376291e-01 9.9633539e-01 9.9835771e-01 ... 1.5251886e-03 6.0232681e-01
 5.6996569e-04]
(array([1., 1., 1., ..., 0., 1., 0.], dtype=float32), array([1., 1., 1., ..., 0., 1., 0.]))
('acc : ', 0.978)
('precision : ', 0.9906054279749478)
('sensitivity : ', 0.9644308943089431)
('specificity : ', 0.9911417322834646)
('MCC : ', 0.9562941291795969)


In [29]:
poidscnn1 = pretrain.get_weights()[0]
bias1 = pretrain.get_weights()[1]
poidscnn2 = pretrain.get_weights()[2]
bias2 = pretrain.get_weights()[3]

In [25]:
# Test  predict rna with prenetwork (avec les données RNA)
print('predicting')            
predictions = pretrain.predict_proba(testing[0])[:,1]
print(predictions)
for i,nulll in enumerate(predictions):
    predictions[i] = round(predictions[i])
print(predictions,true_y)
perfs = calculate_performance(len(predictions), predictions, true_y)
print("acc : ", perfs[0])
print("precision : ", perfs[1])
print("sensitivity : ", perfs[2])
print("specificity : ", perfs[3])
print("MCC : ", perfs[4])

predicting
[4.42874409e-07 6.36836630e-04 2.39901328e-06 ... 4.62607801e-08
 1.06100806e-07 9.82342954e-07]
(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([1, 1, 0.0, ..., 0.0, 1, 1], dtype=object))
('acc : ', 0.50258)
('precision : ', 0.6356877323420075)
('sensitivity : ', 0.006855355997434253)
('specificity : ', 0.996088761174968)
('MCC : ', 0.020123535001122184)


### Training

In [11]:
verif = []
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

verif = pd.read_csv("genomes/negatifs_m-m-str.csv", sep = "\t",header=None)
verif = np.array(verif)
print(len(verif))
for i in range(len(verif)):
    for j in range(2):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            

neg1 = verif

verif=[]

verif = pd.read_csv("genomes/negatifs_divers-str.csv", sep = "\t",header=None)
verif = np.array(verif)
print(len(verif))
for i in range(len(verif)):
    for j in range(2):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            

neg2 = verif
verif=[]

verif = pd.read_csv("genomes/negatifs_mouse_divers-str.csv", sep = "\t",header=None)
verif = np.array(verif)
print(len(verif))
for i in range(len(verif)):
    for j in range(2):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
neg3 = verif
verif=[]


verif = pd.read_csv("genomes/positifs_m-m-str.csv", sep = "\t",header=None)  
verif = np.array(verif)
print(len(verif))
for i in range(len(verif)):
    for j in range(2):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            
pos1 = verif
l = len(pos1)
verif=[]

verif = pd.read_csv("genomes/positifs_divers-str.csv", sep = "\t",header=None)  
verif = np.array(verif)
print(len(verif))
for i in range(len(verif)):
    for j in range(2):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            
pos2 = verif
l=l+len(pos2)
verif=[]

verif = pd.read_csv("genomes/positifs_mouse_divers-str.csv", sep = "\t",header=None)  
verif = np.array(verif)
print(len(verif))
for i in range(len(verif)):
    for j in range(2):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            
pos3 = verif
l=l+len(pos3)
verif=[]

bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3))


pos1 = []
neg1 = []
pos2 = []
neg2 = []

labels = np.zeros((len(bdd),1))

bdd = np.concatenate((bdd,labels),axis=1)
for i in range(l):
    bdd[i,4]=1
labels=[]

26017
24222
37991
26023
24229
37993


In [12]:
len(bdd)

176475

In [13]:
nb_train = 165000
nb_val = 6000
nb_test = 5000 #initial avant retour à 50%-50%

In [14]:
# shuffle total

indices = np.arange(len(bdd))
shuffle(indices)
bdd = bdd[indices]
indices=[]

# 
l = len(bdd)
matrice = np.zeros((l,36,36,1)) #4
matrice1 = np.zeros((l,36,36,1))
matrice2 = np.zeros((l,36,4))
matrice3 = np.zeros((l,36,4))

for i in range(l):
    seq1 = bdd[i,0]
    seq2 = bdd[i,1]
    prob1 = np.array(clean(bdd[i,2]))
    prob2 = np.array(clean(bdd[i,3]))
    for j in range(len(seq1)):
        for k in range(len(seq2)):
            if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                matrice[i,j,k,0] = 1
                #matrice[i,j,k,1] = prob1[j]+prob2[k]
            elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                matrice[i,j,k,0] = 1
                #matrice[i,j,k,1] = prob1[j]+prob2[k]
            elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                matrice[i,j,k,0] = 1
                #matrice[i,j,k,1] = prob1[j]+prob2[k]
            matrice1[i,j,k,0] = prob1[j]+prob2[k]
    for j in range(len(seq1)):
        if seq1[j]=='a':
            matrice2[i,j,0] = 1
        elif seq1[j]=='u':
            matrice2[i,j,1] = 1
        elif seq1[j]=='g':
            matrice2[i,j,2] = 1
        elif seq1[j]=='c':
            matrice2[i,j,3] = 1
    for j in range(len(seq2)):
        if seq2[j]=='a':
            matrice3[i,j,0] = 1
        elif seq2[j]=='u':
            matrice3[i,j,1] = 1
        elif seq2[j]=='g':
            matrice3[i,j,2] = 1
        elif seq2[j]=='c':
            matrice3[i,j,3] = 1                    

In [15]:
#argtest0 = np.argwhere(bdd[-nb_test:,2]==1)[:,0]
#argtest1 = np.argwhere(bdd[-nb_test:,2]==0)[:,0][:len(argtest0)]
#argtest=np.concatenate((argtest0,argtest1))+len(bdd)-nb_test
#argtest0=[]
#argtest1=[]

In [16]:
#np.sum(bdd[argtest,2])/len(argtest)

In [17]:
training = []
training.append(matrice[:nb_train])
training.append(matrice1[:nb_train])
training.append(matrice2[:nb_train])
training.append(matrice3[:nb_train])

validation = []
validation.append(matrice[nb_train:nb_train+nb_val])
validation.append(matrice1[nb_train:nb_train+nb_val])
validation.append(matrice2[nb_train:nb_train+nb_val])
validation.append(matrice3[nb_train:nb_train+nb_val])

testing = []
testing.append(matrice[-nb_test:])
testing.append(matrice1[-nb_test:])
testing.append(matrice2[-nb_test:])
testing.append(matrice3[-nb_test:])

labels = bdd[:,4]
bdd = []
y = labels[:nb_train]
y = keras.utils.np_utils.to_categorical(y,2)
val_y = labels[nb_train:nb_train+nb_val]
val_y = keras.utils.np_utils.to_categorical(val_y,2)
true_y = labels[-nb_test:]
argtest=[]

In [18]:
np.sum(y[:,1])

82487.0

In [24]:
m = merge_networks_train_predict_doublemerge2()

('training', 165000)
configure cnn network
(None, 10, 10, 24)
(None, 4, 4, 6)
configure cnn network
(None, 10, 10, 24)
(None, 4, 4, 6)
(None, 5, 64)
(None, 5, 64)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


model training
Train on 165000 samples, validate on 6000 samples
Epoch 1/6
165000/165000 [==============================] - 160s 972us/step - loss: 0.5952 - val_loss: 0.5607
Epoch 2/6
165000/165000 [==============================] - 159s 962us/step - loss: 0.5473 - val_loss: 0.5262
Epoch 3/6
165000/165000 [==============================] - 159s 963us/step - loss: 0.5250 - val_loss: 0.5283
Epoch 4/6
165000/165000 [==============================] - 159s 962us/step - loss: 0.5096 - val_loss: 0.5230
Epoch 5/6
165000/165000 [==============================] - 162s 982us/step - loss: 0.4941 - val_loss: 0.5046
Epoch 6/6
165000/165000 [==============================] - 162s 980us/step - loss: 0.4798 - val_loss: 0.5000
predicting
[0.92890245 0.9656374  0.30589166 ... 0.03495842 0.27011627 0.8712664 ]
(array([1., 1., 0., ..., 0., 0., 1.], dtype=float32), array([1, 1, 1, ..., 0.0, 0.0, 0.0], dtype=object))
('acc : ', 0.7554)
('precision : ', 0.7743929359823399)
('sensitivity : ', 0.711273317112733

model training
Train on 165000 samples, validate on 6000 samples
Epoch 1/6
165000/165000 [==============================] - 160s 972us/step - loss: 0.5952 - val_loss: 0.5607
Epoch 2/6
165000/165000 [==============================] - 159s 962us/step - loss: 0.5473 - val_loss: 0.5262
Epoch 3/6
165000/165000 [==============================] - 159s 963us/step - loss: 0.5250 - val_loss: 0.5283
Epoch 4/6
165000/165000 [==============================] - 159s 962us/step - loss: 0.5096 - val_loss: 0.5230
Epoch 5/6
165000/165000 [==============================] - 162s 982us/step - loss: 0.4941 - val_loss: 0.5046
Epoch 6/6
165000/165000 [==============================] - 162s 980us/step - loss: 0.4798 - val_loss: 0.5000
predicting
[0.92890245 0.9656374  0.30589166 ... 0.03495842 0.27011627 0.8712664 ]
(array([1., 1., 0., ..., 0., 0., 1.], dtype=float32), array([1, 1, 1, ..., 0.0, 0.0, 0.0], dtype=object))
##### ('acc : ', 0.7554)
('precision : ', 0.7743929359823399)
('sensitivity : ', 0.7112733171127331)
('specificity : ', 0.7983425414364641)
('MCC : ', 0.5118350316396348)
dense-merge2 512 dense-merge1 128 dense 512-128-64-2 cnn24/6 avg3 cnn64,7 max6

In [23]:
def merge_networks_train_predict_doublemerge2():
    
    print('training', nb_train)
    training_net=[]
    training_net1 = []
    training_net2 = []
    training_net1.append(get_cnn_network3())
    training_net1.append(get_cnn_network3())
    training_net2.append(get_cnn_network_seq2())
    training_net2.append(get_cnn_network_seq2())
    
    
    model2 = Sequential()
    model2.add(Merge(training_net2, mode ='concat'))
    model2.add(Dense(512,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model2.add(BatchNormalization())
    model2.add(Activation('relu'))
    #model2.add(Dropout(0.1))
    model1 = Sequential()
    model1.add(Merge(training_net1, mode ='concat'))
    model1.add(Dense(128,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    #model1.add(Dropout(0.1))
    model1.add(BatchNormalization())
    model1.add(Activation('relu'))
    training_net.append(model1)
    training_net.append(model2)
    model = Sequential()
    model.add(Merge(training_net, mode='concat'))
    
    model.add(Dense(512,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(128,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(64,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
        
    earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
    
    
    print('model training')
    model.fit(training, y, batch_size=16, epochs=6, verbose=1, validation_data=(validation, val_y), callbacks=[earlystopper])
    
    # test   
    
    print('predicting')
    
        
    predictions = model.predict_proba(testing)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])
    
    
    return model

In [20]:
def get_cnn_network3():    
    k = 6
    nbfilter = 24
    # init_weights
    I = np.eye(k)
    M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
    I2 = np.zeros((k,k))
    M2 = np.zeros((k,k))
    for j in range(k):
        I2[:,j] = I[:,k-j-1]
        M2[:,j] = M[:,k-j-1]        
    W = np.zeros((k,k,1,nbfilter))
    W[:,:,0,0] = I
    #W[:,:,1,0] = I
    W[:,:,0,1] = I2
   # W[:,:,1,1] = I2
    W[:,:,0,2] = M
   # W[:,:,1,2] = M
    W[:,:,0,3] = M2
   # W[:,:,1,3] = M2
    for j in range(4,nbfilter):
        W[:,:,0,j] = np.random.randn(k,k)*0.2
       # W[:,:,1,j] = np.random.randn(k,k)*0.2
    # cnn
    print('configure cnn network')
 
    model = Sequential()
    #model.add(Conv2D(filters = nbfilter, kernel_size=(k,k), padding='valid', input_shape=(36,36,2),strides=(1,1),weights=[poidscnn1,bias1]))
    model.add(Conv2D(filters = nbfilter, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter)]))    
    #model.add(BatchNormalization())    
    #model.add(Activation('sigmoid'))
    #model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(3,3)))
    print(model.output_shape)
    
    nbfilter2 = 6
    k2 = 7
    #
    I = np.eye(k2)
    M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
    I2=np.zeros((k2,k2))
    M2=np.zeros((k2,k2))
    for j in range(k2):
        I2[:,j] = I[:,k2-j-1]
        M2[:,j] = M[:,k2-j-1]   
    Z = np.zeros((k2,k2,nbfilter,nbfilter2))
    for u in range(nbfilter):
        Z[:,:,u,0] = I
        Z[:,:,u,1] = I2
        Z[:,:,u,2] = M
        Z[:,:,u,3] = M2
        for p in range(4,nbfilter2):
            Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            
    #
    #model.add(Conv2D(filters = nbfilter2, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[poidscnn2,bias2]))
    model.add(Conv2D(filters = nbfilter2, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter2)]))
    #model.add(BatchNormalization())

    print(model.output_shape)
    
    model.add(Flatten())
        
    return model

In [21]:
def get_cnn_network_seq2():
    f = 7
    nb = 64
        
    model = Sequential()
    model.add(Conv1D(filters = nb, kernel_size = f, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None)))
    #model.add(BatchNormalization())
    #model.add(Activation('sigmoid'))
    model.add(MaxPooling1D(pool_size=6))
    print(model.output_shape)
    model.add(Flatten())
    return model

('acc : ', 0.6827445652173914)
('precision : ', 0.6881118881118881)
('sensitivity : ', 0.6684782608695652)
('specificity : ', 0.6970108695652174)
('MCC : ', 0.3656379954692535)

Deux couches pour cnn1 24/4 dense512 pour merge1 puis dense128-2

In [120]:
def get_cnn_network_seq():
    f = 7
    nb = 128
    model = Sequential()
    model.add(Conv1D(filters = nb, kernel_size = f, strides = 1, padding = 'valid', input_shape=(35,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    #model.add(Activation('sigmoid'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Flatten())
    return model

In [128]:
def merge_networks_train_predict():
    
    print('training', nb_train)
    
    #hid = 64*8*8
    training_net = []
    training_net.append(get_cnn_network2())
    training_net.append(get_cnn_network_seq())
    training_net.append(get_cnn_network_seq())

    #y, encoder = preprocess_labels(training_label)
    #val_y, encoder = preprocess_labels(validation_label, encoder = encoder)
       
    
    model = Sequential()
    model.add(Merge(training_net, mode ='concat'))
    #model.add(BatchNormalization())
    
    #print(hid)
    model.add(Dense(256,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(256,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    
    #checkpointer = ModelCheckpoint(filepath="models/bestmodel.hdf5", verbose=0, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    #validation_data=(np.transpose(validmat['validxdata'],axes=(0,2,1)), validmat['validdata']), callbacks=[checkpointer,earlystopper]
    print('model training')
    model.fit(training, y, batch_size=64, epochs=15, verbose=0, validation_data=(validation, val_y), callbacks=[earlystopper])
    
    # test   
    
    print('predicting')
    
        
    predictions = model.predict_proba(testing)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

    
    return model

In [22]:
def get_cnn_network2():    
    k = 5
    nbfilter = 24
    # init_weights
    I = np.eye(k)
    M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
    I2 = np.zeros((k,k))
    M2 = np.zeros((k,k))
    for j in range(k):
        I2[:,j] = I[:,k-j-1]
        M2[:,j] = M[:,k-j-1]        
    W = np.zeros((k,k,1,nbfilter))
    W[:,:,0,0] = I
    W[:,:,0,1] = I2
    W[:,:,0,2] = M
    W[:,:,0,3] = M2
    for j in range(4,nbfilter):
        W[:,:,0,j] = np.random.randn(k,k)*0.2        
    # cnn
    print('configure cnn network')
 
    model = Sequential()
    model.add(Conv2D(filters = nbfilter, kernel_size=(k,k), padding='valid', input_shape=(35,35,1),strides=(1,1),weights=[W,np.zeros(nbfilter)]))
    model.add(BatchNormalization())
    #model.add(Activation('sigmoid'))
    #model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(3,3)))
    print(model.output_shape)
    
    model.add(Flatten())
        
    return model

In [28]:
print(training[0][0,:24,:24,0])

[[0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.

In [54]:
print(training[0][3,12:,:24,0])

[[0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 0. 0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1.]
 [1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0.

In [27]:
y[:10]

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

## Visualisation données

In [40]:
np.set_printoptions(threshold=35*35)

In [55]:
a=2
print((matrice_train[a,:,:,0]))[:24,:24]

[[1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0.

In [56]:
y[:5]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

### ARCHIVES

### Récupération rapide des séquences à partir de chr_nb start end

In [ ]:
# download genome at ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_human/release_24/GRCh38.primary_assembly.genome.fa.gz

parser = SeqIO.parse(open("genomes/GRCh38.genome.fa"),"fasta")

dict_fasta = dict([(seq.id, seq) for seq in parser])

ide, begin, end = ['chr17',8222876,8222895]

dict_fasta[ide][begin:end].format("fasta").split('\n')[1]

### A partir de là c'est l'ancienne méthode (à conserver au cas où, archive)

#### On  sauvegarde la base de données dans un fichier csv

In [28]:
browser = webdriver.Chrome()
import csv

writer = csv.writer(open("genomes/database.csv", 'w',newline=''),delimiter="\t")
for i,el in enumerate(varrbdd):
    row = transformer_database2(varrbdd[i])    
    writer.writerow(row)

On cherche les trous dans les données

In [7]:
bdd_5000_trouee = pd.read_csv("genomes/database.csv", sep = "\t",header = None)

In [8]:
bdd_5000_trouee = np.array(bdd_5000_trouee)
l = []
for i,el in enumerate(bdd_5000_trouee):
    if type(bdd_5000_trouee[i,0])==float or type(bdd_5000_trouee[i,1])==float:
        a=0
    elif len(bdd_5000_trouee[i,0])==0 or len(bdd_5000_trouee[i,0])==0:
        a=0
    else:
        l.append(i)
l = np.array(l)
len(l)

4983

In [23]:
bdd_miRNA_mRNA = bdd_5000_trouee[l]

In [26]:
bdd_miRNA_mRNA = nettoyer(bdd_miRNA_mRNA)

In [29]:
bdd_miRNA_mRNA = traduire_séquences(bdd_miRNA_mRNA)

In [20]:
bdd_apprentissage = construire_seq_et_structure_bdd(bdd_miRNA_mRNA[:15])

#### A partir de mainenant, on essaye d'obtenir les probabilités d'accesibilité de chaque nucléotide sur une séquence quelconque ARN par exemple 'ACGUUGUCACACGAU'

In [10]:
def traduire_test(seq,strand):
    nucleotides = ''
    if strand:
        string = ''
        for i in seq:
            string = i + string
        seq = string
        for lettre in seq:
            if lettre=='a':
                nucleotides=nucleotides+'u'
            elif lettre=='c':
                nucleotides=nucleotides+'g'
            elif lettre=='g':
                nucleotides=nucleotides+'c'
            else:
                nucleotides=nucleotides+'a'
    else:
        for lettre in seq:
            if lettre=='a':
                nucleotides=nucleotides+'a'
            elif lettre=='c':
                nucleotides=nucleotides+'c'
            elif lettre=='g':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'    
    return nucleotides

In [55]:
traduire_test('gaagacactcttgtctcagtaaaaggtaaaggagggctcgtccgggatttgaacccgggacctctcgcacccgaagcgagaatcatacccctagaccaacgagccgacgtgcggacgttgccgcgaaccgccttagaggtcgtgccaggcttgctgtagtg',True)

'cacuacagcaagccuggcacgaccucuaaggcgguucgcggcaacguccgcacgucggcucguuggucuagggguaugauucucgcuucgggugcgagaggucccggguucaaaucccggacgagcccuccuuuaccuuuuacugagacaagagugucuuc'

In [ ]:
'gaagacactcttgtctcagtaaaaggtaaaggagggctcgtccgggatttgaacccgggacctctcgcacccgaagcgagaatcatacccctagaccaacgagccgacgtgcggacgttgccgcgaaccgccttagaggtcgtgccaggcttgctgtagtg'

In [12]:
bdd = pd.read_csv("rise_human_transcriptome (copie).csv", sep = "\t",header=None)

/media/DATA/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
# bdd convertie panda -> numpy array et seuls les 70 000 premiers RRIs retenus
arrbdd = np.array(bdd)[:70000]

#enlever 'chr' et formater les numéros de chromosomes
for i,el in enumerate(arrbdd[:,0]):
    arrbdd[i,0] = arrbdd[i,0][3:]

for i,el in enumerate(arrbdd[:,0]):
    if arrbdd[i,0] == 'X':
        arrbdd[i,0] = '23'
    if arrbdd[i,0] == 'Y':
        arrbdd[i,0] = '24'
    if arrbdd[i,0] == '1':
        arrbdd[i,0] = '01'
    if arrbdd[i,0] == '2':
        arrbdd[i,0] = '02'
    if arrbdd[i,0] == '3':
        arrbdd[i,0] = '03'
    if arrbdd[i,0] == '4':
        arrbdd[i,0] = '04'
    if arrbdd[i,0] == '5':
        arrbdd[i,0] = '05'
    if arrbdd[i,0] == '6':
        arrbdd[i,0] = '06'
    if arrbdd[i,0] == '7':
        arrbdd[i,0] = '07'
    if arrbdd[i,0] == '8':
        arrbdd[i,0] = '08'
    if arrbdd[i,0] == '9':
        arrbdd[i,0] = '09'

for i,el in enumerate(arrbdd[:,3]):
    arrbdd[i,3] = arrbdd[i,3][3:]

for i,el in enumerate(arrbdd[:,3]):
    if arrbdd[i,3] == 'X':
        arrbdd[i,3] = '23'
    elif arrbdd[i,3] == 'Y':
        arrbdd[i,3] = '24'  
    elif arrbdd[i,3] == '1':
        arrbdd[i,3] = '01'
    elif arrbdd[i,3] == '2':
        arrbdd[i,3] = '02'
    elif arrbdd[i,3] == '3':
        arrbdd[i,3] = '03'
    elif arrbdd[i,3] == '4':
        arrbdd[i,3] = '04'
    elif arrbdd[i,3] == '5':
        arrbdd[i,3] = '05'
    elif arrbdd[i,3] == '6':
        arrbdd[i,3] = '06'
    elif arrbdd[i,3] == '7':
        arrbdd[i,3] = '07'
    elif arrbdd[i,3] == '8':
        arrbdd[i,3] = '08'
    elif arrbdd[i,3] == '9':
        arrbdd[i,3] = '09'


#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'M') & (arrbdd[:,3] != 'M'))
varrbdd = arrbdd[arg_sansM[:,0]]

#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = 'true'
    else:
        varrbdd[i,8] = 'false'

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = 'true'
    else:
        varrbdd[i,9] = 'false'


#int str
for i,el in enumerate(varrbdd[:,1]):
    if type(el) == int:
        varrbdd[i,1] = str(el)
for i,el in enumerate(varrbdd[:,2]):
    if type(el) == int:
        varrbdd[i,2] = str(el)
for i,el in enumerate(varrbdd[:,4]):
    if type(el) == int:
        varrbdd[i,4] = str(el)
for i,el in enumerate(varrbdd[:,5]):
    if type(el) == int:
        varrbdd[i,5] = str(el)

#génération position début et fin de séquence (nucléotides)

for i,el in enumerate(varrbdd[:,1]):
    milieu = int((int(varrbdd[i,1])+int(varrbdd[i,2]))/2)
    varrbdd[i,1] = str(milieu-50)
    varrbdd[i,2] = str(milieu+50)
    milieu = int((int(varrbdd[i,4])+int(varrbdd[i,5]))/2)
    varrbdd[i,4] = str(milieu-50)
    varrbdd[i,5] = str(milieu+50)

# Génération URLs

#for i,el in enumerate(varrbdd[:,6]):
 #   varrbdd[i,6] = "https://www.ncbi.nlm.nih.gov/nuccore/NC_0000"+varrbdd[i,0]+"?report=genbank&from="+varrbdd[i,1]+"&to="+varrbdd[i,2]+"&strand="+varrbdd[i,8]+".html"
#for i,el in enumerate(varrbdd[:,6]):
 #   varrbdd[i,6] = 'https://www.ncbi.nlm.nih.gov/nuccore/NC_0000'+varrbdd[i,0]+'?report=fasta&log$=seqview&format=text&from='+varrbdd[i,1]+'&to='+varrbdd[i,2]

#for i,el in enumerate(varrbdd[:,7]):
 #   varrbdd[i,7] = "https://www.ncbi.nlm.nih.gov/nuccore/NC_0000"+varrbdd[i,3]+"?report=genbank&from="+varrbdd[i,4]+"&to="+varrbdd[i,5]+"&strand="+varrbdd[i,9]+".html"
#for i,el in enumerate(varrbdd[:,7]):
 #   varrbdd[i,7] = 'https://www.ncbi.nlm.nih.gov/nuccore/NC_0000'+varrbdd[i,3]+'?report=fasta&log$=seqview&format=text&from='+varrbdd[i,4]+'&to='+varrbdd[i,5]

In [ ]:
def recuperer_sequence(url):
    browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(4)
    innerHTML = browser.execute_script("return document.body.innerHTML")
    browser.quit()
    soup = BeautifulSoup(innerHTML,"lxml")
    seq = soup.find_all("span",{'class':"ff_line"})
    rep = seq[0].text+seq[1].text+seq[2].text+seq[3].text
    rep = ''.join(rep.split())
    return rep

def recuperer_sequence2(url):
    browser.get(url)
    time.sleep(1.3)
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,"lxml")
    seq = soup.find_all("div",{'class':"seq gbff"})
    rep = seq[0].text   
    return rep

def transformer_database(bddrow):
    sequence=recuperer_sequence(bddrow[6])
    sequence=transformer_sequence(sequence,len(sequence))
    sequencebis=recuperer_sequence(bddrow[7])
    sequencebis=transformer_sequence(sequencebis,len(sequencebis))    
    bddrow = np.array([bddrow[18],bddrow[19],bddrow[20],bddrow[21],sequence,bddrow[0],bddrow[1],bddrow[2],bddrow[8],bddrow[10],bddrow[11],bddrow[14],bddrow[16],sequencebis,bddrow[3],bddrow[4],bddrow[5],bddrow[9],bddrow[12],bddrow[13],bddrow[15],bddrow[17]], dtype=object)
    return bddrow

def transformer_database2(bddrow):
    sequence=recuperer_sequence2(bddrow[6])
    sequencebis=recuperer_sequence2(bddrow[7])
    bddrow = np.array([sequence,sequencebis,bddrow[8],bddrow[9]], dtype=object)
    return bddrow

def construire_seq_et_structure_bdd(bdd):
    for i,el in enumerate(bdd):
        bdd[i,2] = np.array(RNA.pfl_fold_up(bdd[i,0],1,201,201))[1:,1]
        bdd[i,3] = np.array(RNA.pfl_fold_up(bdd[i,1],1,201,201))[1:,1]
        #bdd[i,0] = transformer_sequence(bdd[i,0],201)
        #bdd[i,1] = transformer_sequence(bdd[i,1],201)
    return bdd

def nettoyer(bdd):
    for i,el in enumerate(bdd):
        bdd[i,0]=bdd[i,0].split('\n')[1]+bdd[i,0].split('\n')[2]+bdd[i,0].split('\n')[3]
        bdd[i,1]=bdd[i,1].split('\n')[1]+bdd[i,1].split('\n')[2]+bdd[i,1].split('\n')[3]
    return bdd

def traduire_séquences(bdd):
    for i,el in enumerate(bdd):
        bdd[i,0]=traduire(bdd[i,0],bdd[i,2])
        bdd[i,1]=traduire(bdd[i,1],bdd[i,3])
    return bdd